In [1]:
# 라이브러리 호출
import ast
from ast import literal_eval
import pandas as pd
from argparse import ArgumentParser

import warnings

In [2]:
# 1. 기존 데이터 프레임 불러오기
csv_path = "../data/game_info_igdb.csv"
data = pd.read_csv(csv_path)

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,id,age_ratings,aggregated_rating,aggregated_rating_count,alternative_names,artworks,bundles,category,checksum,collection,...,tags,themes,total_rating,total_rating_count,updated_at,url,version_parent,version_title,videos,websites
0,149552,NaN,NaN,NaN,NaN,[17661],NaN,0,c4681559-ce23-46ca-43c3-c96b02080d9b,NaN,...,"[1, 18, 19, 268435487, 268435488, 536870984]","[1, 18, 19]",NaN,NaN,1709145898,https://www.igdb.com/games/biolum,NaN,NaN,[82544],"[188558, 457253]"
1,299159,NaN,NaN,NaN,NaN,NaN,NaN,0,85e11c7c-611c-ea2f-cd1b-83837fe14540,NaN,...,NaN,[17],NaN,NaN,1714002923,https://www.igdb.com/games/witch--2,NaN,NaN,NaN,[633585]
2,202326,NaN,NaN,NaN,NaN,NaN,NaN,0,275a30ba-7ae8-4b8e-53be-ab659e9a4cad,NaN,...,"[268435487, 536898737]",NaN,NaN,NaN,1711066578,https://www.igdb.com/games/repentance,NaN,NaN,NaN,"[285224, 492342]"
3,172900,NaN,NaN,NaN,NaN,[52904],NaN,0,2f545826-d2a0-1501-05ad-354aa9424c23,NaN,...,"[268435471, 268435488]",NaN,NaN,NaN,1712115898,https://www.igdb.com/games/hidden-object-hunt-...,NaN,NaN,NaN,"[229167, 470416]"
4,7428,NaN,NaN,NaN,"[136735, 136736]",NaN,NaN,0,9fea5d46-7723-9aff-9236-a6822defe264,NaN,...,"[1, 268435489, 536872733, 536872739, 536881061...",[1],58.426966,5.0,1709857277,https://www.igdb.com/games/tank,NaN,NaN,[15992],"[57579, 338886]"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276891 entries, 0 to 276890
Data columns (total 58 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       276891 non-null  int64  
 1   age_ratings              61625 non-null   object 
 2   aggregated_rating        15355 non-null   float64
 3   aggregated_rating_count  15355 non-null   float64
 4   alternative_names        61728 non-null   object 
 5   artworks                 110151 non-null  object 
 6   bundles                  12921 non-null   object 
 7   category                 276891 non-null  int64  
 8   checksum                 276891 non-null  object 
 9   collection               33250 non-null   float64
 10  collections              38159 non-null   object 
 11  cover                    212034 non-null  float64
 12  created_at               276891 non-null  int64  
 13  dlcs                     2992 non-null    object 
 14  expa

In [5]:
# 2. 필요한 컬럼 불러오기
cols = ['id', 'name', 'platforms', 'status', 'rating', 'genres', 'themes', 'keywords', 'similar_games']
games_df = data[cols]
games_df.head()

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
0,149552,Biolum,"[6, 161, 163, 385]",NaN,NaN,"[31, 32]","[1, 18, 19]",[72],"[25311, 25646, 26658, 55282, 68271, 96217, 103..."
1,299159,Witch,[6],NaN,NaN,"[8, 9, 31, 32]",[17],"[1180, 2058, 23548]",NaN
2,202326,Repentance,[6],NaN,NaN,[31],NaN,[27825],"[87622, 280471, 280472, 281229, 286130, 286131..."
3,172900,Hidden Object Hunt Classic,[6],NaN,NaN,"[15, 32]",NaN,NaN,"[19404, 31515, 33603, 33646, 36269, 55590, 658..."
4,7428,Tank,[52],NaN,58.426966,[33],[1],"[1821, 1827, 10149, 38272]","[15851, 25640, 43367, 56033, 87728, 103369, 11..."


In [6]:
# 3. 결측치 처리 - (1) similar_games, platforms, rating 결측치인 데이터 drop
test_df = games_df.dropna(subset=['similar_games'])
test_df = test_df.dropna(subset=['platforms'])
test_df = test_df.dropna(subset=['rating'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28889 entries, 4 to 276886
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             28889 non-null  int64  
 1   name           28889 non-null  object 
 2   platforms      28889 non-null  object 
 3   status         2533 non-null   float64
 4   rating         28889 non-null  float64
 5   genres         28564 non-null  object 
 6   themes         22244 non-null  object 
 7   keywords       18241 non-null  object 
 8   similar_games  28889 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.2+ MB


In [7]:
# 3. 결측치 처리 - (2) genres,themes,kewyrods가 전부 결측치인 데이터 drop
all_nan_indices = test_df[test_df['genres'].isna() & test_df['themes'].isna() & test_df['keywords'].isna()].index
test_df = test_df.drop(all_nan_indices)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28886 entries, 4 to 276886
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             28886 non-null  int64  
 1   name           28886 non-null  object 
 2   platforms      28886 non-null  object 
 3   status         2533 non-null   float64
 4   rating         28886 non-null  float64
 5   genres         28564 non-null  object 
 6   themes         22244 non-null  object 
 7   keywords       18241 non-null  object 
 8   similar_games  28886 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.2+ MB


In [8]:
# 3. 결측치 처리 - (3) status 결측치 0으로 처리(released로 가정)
test_df['status'] = test_df['status'].fillna(0)
test_df

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
4,7428,Tank,[52],0.0,58.426966,[33],[1],"[1821, 1827, 10149, 38272]","[15851, 25640, 43367, 56033, 87728, 103369, 11..."
9,20979,Fossil Fighters: Champions,[20],0.0,86.000000,[12],NaN,"[9927, 9951]","[19404, 35737, 49414, 56036, 59427, 68393, 874..."
14,19530,Quake Champions,[6],4.0,72.575308,[5],"[1, 18]","[114, 129, 453, 524, 546, 575, 1075, 1097, 129...","[564, 7350, 11544, 11646, 17447, 18968, 25657,..."
40,10553,Monster Truck Destruction,"[6, 14]",0.0,50.000000,[10],[1],NaN,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
54,9323,The Guild 2: Pirates of the European Seas,[6],0.0,50.000000,"[12, 13, 15]","[22, 33]","[4, 151, 994, 5657, 8146]","[18623, 23345, 26574, 31515, 35371, 36346, 365..."
...,...,...,...,...,...,...,...,...,...
276840,100600,Nepenthe,"[3, 6, 14]",0.0,70.000000,"[12, 31, 32]","[1, 19, 27]","[220, 1721, 2814]","[25311, 25646, 28309, 35994, 80916, 96217, 105..."
276845,1355,Jet Force Gemini,[4],0.0,80.457890,"[5, 8, 31]","[1, 18]","[575, 660, 947, 1033, 1186, 1289, 1898, 1935, ...","[1877, 18290, 19249, 19564, 28070, 47823, 5503..."
276846,48133,Mario's Time Machine,"[6, 13]",0.0,21.495292,"[9, 15]",[34],[26906],"[236, 27792, 87056, 95776, 109129, 206719, 242..."
276867,89535,Eden - World Builder,[39],0.0,80.000000,"[12, 13, 31]",[35],NaN,"[25311, 26574, 28129, 29348, 35994, 87485, 962..."


In [9]:
# 4. id -> slug 변환 - (1) 문자열을 리스트로 변환
def str_to_list(x):
    try:
        if type(x) == str:
            return literal_eval(x)
        elif type(x) == list:
            return x
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return ""

In [10]:
print("변환 전 형식",type(test_df['platforms'].values[0]))

test_df['platforms'] = test_df['platforms'].apply(lambda x: str_to_list(x))
test_df['genres'] = test_df['genres'].apply(lambda x: str_to_list(x))
test_df['themes'] = test_df['themes'].apply(lambda x: str_to_list(x))
test_df['keywords'] = test_df['keywords'].apply(lambda x: str_to_list(x))
test_df['similar_games'] = test_df['similar_games'].apply(lambda x: str_to_list(x))

print("변환 후 형식",type(test_df['platforms'].values[0]))

변환 전 형식 <class 'str'>
변환 후 형식 <class 'list'>


In [11]:
# 4. id -> name/slug 변환 - (2) 딕셔너리 만들 파일 불러오기
endpoints = '../data/endpoints/'
genres_df = pd.read_csv(endpoints + 'genres.csv')
themes_df = pd.read_csv(endpoints + 'themes.csv')
keywords_df = pd.read_csv(endpoints + 'keywords.csv')
platforms_df = pd.read_csv(endpoints + 'platforms.csv')

In [12]:
# 4. id -> name/slug 변환 - (3) 딕셔너리 만들기
def endpoints_dict(data,column):
  dictionary = dict(zip(data.id, data[column]))
  return dictionary

genres_dict = endpoints_dict(genres_df,'slug')
keywords_dict = endpoints_dict(keywords_df,'slug')
themes_dict = endpoints_dict(themes_df,'slug')
platforms_dict = endpoints_dict(platforms_df,'name')

In [13]:
# 4. id -> name/slug 변환 - (4) 변환하기
def id_to_slug(x, dict):
    try:
        if type(x) == list:
            return [dict[i] for i in x]
        else:
            return ""
    except KeyError:
        # If a KeyError occurs, simply return None
        return None

test_df['genres'] = test_df['genres'].apply(lambda x: id_to_slug(x, genres_dict))
test_df['themes'] = test_df['themes'].apply(lambda x: id_to_slug(x, themes_dict))
test_df['keywords'] = test_df['keywords'].apply(lambda x: id_to_slug(x, keywords_dict))
test_df['platforms'] = test_df['platforms'].apply(lambda x: id_to_slug(x, platforms_dict))

test_df.head()

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
4,7428,Tank,[Arcade],0.0,58.426966,[arcade],[action],"[tank, tank-combat, tank-controls, tank-game]","[15851, 25640, 43367, 56033, 87728, 103369, 11..."
9,20979,Fossil Fighters: Champions,[Nintendo DS],0.0,86.000000,[role-playing-rpg],,"[dead-multiplayer, e3-2011]","[19404, 35737, 49414, 56036, 59427, 68393, 874..."
14,19530,Quake Champions,[PC (Microsoft Windows)],4.0,72.575308,[shooter],"[action, science-fiction]","[arena, bloody, first-person-shooter, fast-pac...","[564, 7350, 11544, 11646, 17447, 18968, 25657,..."
40,10553,Monster Truck Destruction,"[PC (Microsoft Windows), Mac]",0.0,50.000000,[racing],[action],,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
54,9323,The Guild 2: Pirates of the European Seas,[PC (Microsoft Windows)],0.0,50.000000,"[role-playing-rpg, simulator, strategy]","[historical, sandbox]","[pirates, medieval, management, gamersgate, na...","[18623, 23345, 26574, 31515, 35371, 36346, 365..."


In [14]:
def list_to_string(lst):
    return ' '.join(lst)

# apply 함수를 사용하여 리스트를 스트링으로 변환
final_df = test_df.reset_index(drop=True)
final_df['genres'] = final_df['genres'].apply(list_to_string)
final_df['themes'] = final_df['themes'].apply(list_to_string)
final_df['keywords'] = final_df['keywords'].apply(list_to_string)
final_df

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
0,7428,Tank,[Arcade],0.0,58.426966,arcade,action,tank tank-combat tank-controls tank-game,"[15851, 25640, 43367, 56033, 87728, 103369, 11..."
1,20979,Fossil Fighters: Champions,[Nintendo DS],0.0,86.000000,role-playing-rpg,,dead-multiplayer e3-2011,"[19404, 35737, 49414, 56036, 59427, 68393, 874..."
2,19530,Quake Champions,[PC (Microsoft Windows)],4.0,72.575308,shooter,action science-fiction,arena bloody first-person-shooter fast-paced p...,"[564, 7350, 11544, 11646, 17447, 18968, 25657,..."
3,10553,Monster Truck Destruction,"[PC (Microsoft Windows), Mac]",0.0,50.000000,racing,action,,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
4,9323,The Guild 2: Pirates of the European Seas,[PC (Microsoft Windows)],0.0,50.000000,role-playing-rpg simulator strategy,historical sandbox,pirates medieval management gamersgate naval-w...,"[18623, 23345, 26574, 31515, 35371, 36346, 365..."
...,...,...,...,...,...,...,...,...,...
28881,100600,Nepenthe,"[Linux, PC (Microsoft Windows), Mac]",0.0,70.000000,role-playing-rpg adventure indie,action horror comedy,weird surreal hand-drawn,"[25311, 25646, 28309, 35994, 80916, 96217, 105..."
28882,1355,Jet Force Gemini,[Nintendo 64],0.0,80.457890,shooter platform adventure,action science-fiction,robots run-and-gun spaceship action-adventure ...,"[1877, 18290, 19249, 19564, 28070, 47823, 5503..."
28883,48133,Mario's Time Machine,"[PC (Microsoft Windows), DOS]",0.0,21.495292,puzzle strategy,educational,retroachievements,"[236, 27792, 87056, 95776, 109129, 206719, 242..."
28884,89535,Eden - World Builder,[iOS],0.0,80.000000,role-playing-rpg simulator adventure,kids,,"[25311, 26574, 28129, 29348, 35994, 87485, 962..."


In [15]:
# 데이터 축소 - platform("PC (Microsoft Windows)"), status(released)
released_df = final_df[final_df['status']==0]
released_df

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
0,7428,Tank,[Arcade],0.0,58.426966,arcade,action,tank tank-combat tank-controls tank-game,"[15851, 25640, 43367, 56033, 87728, 103369, 11..."
1,20979,Fossil Fighters: Champions,[Nintendo DS],0.0,86.000000,role-playing-rpg,,dead-multiplayer e3-2011,"[19404, 35737, 49414, 56036, 59427, 68393, 874..."
3,10553,Monster Truck Destruction,"[PC (Microsoft Windows), Mac]",0.0,50.000000,racing,action,,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
4,9323,The Guild 2: Pirates of the European Seas,[PC (Microsoft Windows)],0.0,50.000000,role-playing-rpg simulator strategy,historical sandbox,pirates medieval management gamersgate naval-w...,"[18623, 23345, 26574, 31515, 35371, 36346, 365..."
5,30417,Steel Invaders,[PC (Microsoft Windows)],0.0,50.000000,indie,action,,"[32902, 37419, 40524, 76263, 96217, 105233, 10..."
...,...,...,...,...,...,...,...,...,...
28881,100600,Nepenthe,"[Linux, PC (Microsoft Windows), Mac]",0.0,70.000000,role-playing-rpg adventure indie,action horror comedy,weird surreal hand-drawn,"[25311, 25646, 28309, 35994, 80916, 96217, 105..."
28882,1355,Jet Force Gemini,[Nintendo 64],0.0,80.457890,shooter platform adventure,action science-fiction,robots run-and-gun spaceship action-adventure ...,"[1877, 18290, 19249, 19564, 28070, 47823, 5503..."
28883,48133,Mario's Time Machine,"[PC (Microsoft Windows), DOS]",0.0,21.495292,puzzle strategy,educational,retroachievements,"[236, 27792, 87056, 95776, 109129, 206719, 242..."
28884,89535,Eden - World Builder,[iOS],0.0,80.000000,role-playing-rpg simulator adventure,kids,,"[25311, 26574, 28129, 29348, 35994, 87485, 962..."


In [16]:
pc_df = released_df[released_df['platforms'].apply(lambda x: "PC (Microsoft Windows)" in x)]
pc_df = pc_df.reset_index(drop=True)
pc_df

,id,name,platforms,status,rating,genres,themes,keywords,similar_games
0,10553,Monster Truck Destruction,"[PC (Microsoft Windows), Mac]",0.0,50.000000,racing,action,,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
1,9323,The Guild 2: Pirates of the European Seas,[PC (Microsoft Windows)],0.0,50.000000,role-playing-rpg simulator strategy,historical sandbox,pirates medieval management gamersgate naval-w...,"[18623, 23345, 26574, 31515, 35371, 36346, 365..."
2,30417,Steel Invaders,[PC (Microsoft Windows)],0.0,50.000000,indie,action,,"[32902, 37419, 40524, 76263, 96217, 105233, 10..."
3,9236,Top Gun: Combat Zones,"[PC (Microsoft Windows), PlayStation 2, Ninten...",0.0,49.000000,simulator,warfare,combat-flight-simulator flight-simulator fligh...,"[14758, 22621, 26431, 28562, 31480, 37289, 467..."
4,17790,The Treasures of Montezuma 3,"[PC (Microsoft Windows), Mac]",0.0,40.000000,puzzle adventure indie arcade,,steam steam-trading-cards digital-distribution...,"[25222, 25646, 26223, 27266, 55173, 55190, 560..."
...,...,...,...,...,...,...,...,...,...
18932,32495,Fantasy Kingdom Simulator,[PC (Microsoft Windows)],0.0,40.000000,simulator strategy indie,,,"[17130, 17519, 31515, 33603, 33646, 36269, 515..."
18933,3165,Prince of Persia 3D,"[PC (Microsoft Windows), Dreamcast]",0.0,36.372608,adventure,action stealth,parkour death mythology action-adventure egypt...,"[836, 10776, 26950, 28168, 28309, 30245, 36198..."
18934,100600,Nepenthe,"[Linux, PC (Microsoft Windows), Mac]",0.0,70.000000,role-playing-rpg adventure indie,action horror comedy,weird surreal hand-drawn,"[25311, 25646, 28309, 35994, 80916, 96217, 105..."
18935,48133,Mario's Time Machine,"[PC (Microsoft Windows), DOS]",0.0,21.495292,puzzle strategy,educational,retroachievements,"[236, 27792, 87056, 95776, 109129, 206719, 242..."


In [17]:
# 필요없는 컬럼 삭제
pc_df.drop(['platforms', 'status', 'rating'], axis=1, inplace=True)
pc_df

,id,name,genres,themes,keywords,similar_games
0,10553,Monster Truck Destruction,racing,action,,"[57296, 80382, 82090, 87622, 100800, 109292, 1..."
1,9323,The Guild 2: Pirates of the European Seas,role-playing-rpg simulator strategy,historical sandbox,pirates medieval management gamersgate naval-w...,"[18623, 23345, 26574, 31515, 35371, 36346, 365..."
2,30417,Steel Invaders,indie,action,,"[32902, 37419, 40524, 76263, 96217, 105233, 10..."
3,9236,Top Gun: Combat Zones,simulator,warfare,combat-flight-simulator flight-simulator fligh...,"[14758, 22621, 26431, 28562, 31480, 37289, 467..."
4,17790,The Treasures of Montezuma 3,puzzle adventure indie arcade,,steam steam-trading-cards digital-distribution...,"[25222, 25646, 26223, 27266, 55173, 55190, 560..."
...,...,...,...,...,...,...
18932,32495,Fantasy Kingdom Simulator,simulator strategy indie,,,"[17130, 17519, 31515, 33603, 33646, 36269, 515..."
18933,3165,Prince of Persia 3D,adventure,action stealth,parkour death mythology action-adventure egypt...,"[836, 10776, 26950, 28168, 28309, 30245, 36198..."
18934,100600,Nepenthe,role-playing-rpg adventure indie,action horror comedy,weird surreal hand-drawn,"[25311, 25646, 28309, 35994, 80916, 96217, 105..."
18935,48133,Mario's Time Machine,puzzle strategy,educational,retroachievements,"[236, 27792, 87056, 95776, 109129, 206719, 242..."


In [18]:
pc_df.to_csv("../data/final_igdb_pc.csv")